In [64]:
import pandas as pd
import numpy as np
import gc
import time
from contextlib import contextmanager
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import roc_auc_score, roc_curve, f1_score
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import category_encoders as ce
import pickle
from sklearn.svm import SVC
warnings.simplefilter(action='ignore', category=FutureWarning)

In [65]:
train = pd.read_csv("../data/sample_train.txt", delimiter="\t")
test = pd.read_csv("../data/test_id.txt", delimiter="\t")

In [66]:
df = pd.concat([train, test], axis=0)

In [67]:
risk = pd.read_csv("../features/risk/risk.csv")
df = df.merge(risk, on="id", how="left")

In [68]:
symbol = pd.read_csv("../features/symbol/symbol.csv")
df = df.merge(symbol, on="id", how="left")

In [69]:
# dp_emb = pd.read_csv("../features/graph/graph_filtered_64.emb")
# df = df.merge(dp_emb, on="id", how="left")

In [70]:
dp_emb = pd.read_csv("../features/graph/deepwalk_192_filtered.emb")
df = df.merge(dp_emb, on="id", how="left")

In [71]:
dp_emb.shape

(28442, 193)

In [72]:
# nv_emb = pd.read_csv("../data/edge/node2vec_64.emb")
# df = df.merge(nv_emb, on="id", how="left")

In [73]:
graph_info = pd.read_csv("../features/graph/graph_info.csv")
df = df.merge(graph_info, on="id", how="left")

In [74]:
# app_pca = pd.read_csv("../data/app/app_pca_16.csv")
# df = df.merge(app_pca, on="id", how="left")

app_lda = pd.read_csv("../features/app/app_lda_16.csv")
df = df.merge(app_lda, on="id", how="left")

# app_lda = pd.read_csv("../data1/app/app_lda_16.csv")
# df = df.merge(app_lda, on="id", how="left")

# app_nmf = pd.read_csv("../data/app/app_nmf_16.csv")
# df = df.merge(app_nmf, on="id", how="left")

In [75]:
# app_info = pd.read_csv("../features/app/app_info.csv")
# df = df.merge(app_info, on="id", how="left")

app_graph

In [76]:
# to_pca = pd.read_csv("../features/app_graph/to_app_pca_mean.csv")
# from_pca = pd.read_csv("../features/app_graph/from_app_pca_mean.csv")

# df = df.merge(to_pca, on="id", how="left")
# df = df.merge(from_pca, on="id", how="left")

to_pca = pd.read_csv("../features/app_graph/to_app_pca_num_mean.csv")
from_pca = pd.read_csv("../features/app_graph/from_app_pca_num_mean.csv")

df = df.merge(to_pca, on="id", how="left")
df = df.merge(from_pca, on="id", how="left")

to_pca = pd.read_csv("../features/app_graph/to_app_pca_weight_mean.csv")
from_pca = pd.read_csv("../features/app_graph/from_app_pca_weight_mean.csv")

df = df.merge(to_pca, on="id", how="left")
df = df.merge(from_pca, on="id", how="left")

In [77]:
# to_pca_nmf = pd.read_csv("../features/app_graph/to_app_nmf_mean.csv")
# from_pca_nmf = pd.read_csv("../features/app_graph/from_app_nmf_mean.csv")

# df = df.merge(to_pca_nmf, on="id", how="left")
# df = df.merge(from_pca_nmf, on="id", how="left")

# to_pca_nmf = pd.read_csv("../features/app_graph/to_app_nmf_num_mean.csv")
# from_pca_nmf = pd.read_csv("../features/app_graph/from_app_nmf_num_mean.csv")

# df = df.merge(to_pca_nmf, on="id", how="left")
# df = df.merge(from_pca_nmf, on="id", how="left")

# to_pca_nmf = pd.read_csv("../features/app_graph/to_app_nmf_weight_mean.csv")
# from_pca_nmf = pd.read_csv("../features/app_graph/from_app_nmf_weight_mean.csv")

# df = df.merge(to_pca_nmf, on="id", how="left")
# df = df.merge(from_pca_nmf, on="id", how="left")

In [78]:
# to_lda = pd.read_csv("../features/app_graph/to_app_lda_mean.csv")
# from_lda = pd.read_csv("../features/app_graph/from_app_lda_mean.csv")

# df = df.merge(to_lda, on="id", how="left")
# df = df.merge(from_lda, on="id", how="left")

# to_lda = pd.read_csv("../features/app_graph/to_app_lda_num_mean.csv")
# from_lda = pd.read_csv("../features/app_graph/from_app_lda_num_mean.csv")

# df = df.merge(to_lda, on="id", how="left")
# df = df.merge(from_lda, on="id", how="left")

# to_lda = pd.read_csv("../features/app_graph/to_app_lda_weight_mean.csv")
# from_lda = pd.read_csv("../features/app_graph/from_app_lda_weight_mean.csv")

# df = df.merge(to_lda, on="id", how="left")
# df = df.merge(from_lda, on="id", how="left")

risk_graph

In [79]:
# to_risk = pd.read_csv("../features/risk_graph/to_risk_mean.csv")
# from_risk = pd.read_csv("../features/risk_graph/from_risk_mean.csv")

# df = df.merge(to_risk, on="id", how="left")
# df = df.merge(from_risk, on="id", how="left")

to_risk = pd.read_csv("../features/risk_graph/to_risk_num_mean.csv")
from_risk = pd.read_csv("../features/risk_graph/from_risk_num_mean.csv")

df = df.merge(to_risk, on="id", how="left")
df = df.merge(from_risk, on="id", how="left")

to_risk = pd.read_csv("../features/risk_graph/to_risk_weight_mean.csv")
from_risk = pd.read_csv("../features/risk_graph/from_risk_weight_mean.csv")

df = df.merge(to_risk, on="id", how="left")
df = df.merge(from_risk, on="id", how="left")

# to_risk = pd.read_csv("../features/risk_graph/to_risk_sum.csv")
# from_risk = pd.read_csv("../features/risk_graph/from_risk_sum.csv")

# df = df.merge(to_risk, on="id", how="left")
# df = df.merge(from_risk, on="id", how="left")

to_risk = pd.read_csv("../features/risk_graph/to_risk_num_sum.csv")
from_risk = pd.read_csv("../features/risk_graph/from_risk_num_sum.csv")

df = df.merge(to_risk, on="id", how="left")
df = df.merge(from_risk, on="id", how="left")

to_risk = pd.read_csv("../features/risk_graph/to_risk_weight_sum.csv")
from_risk = pd.read_csv("../features/risk_graph/from_risk_weight_sum.csv")

df = df.merge(to_risk, on="id", how="left")
df = df.merge(from_risk, on="id", how="left")

symbol_graph

In [80]:
# to_cat_num = pd.read_csv("../features/symbol_graph/to_symbol_sum.csv")
# from_cat_num = pd.read_csv("../features/symbol_graph/from_symbol_sum.csv")

# df = df.merge(to_cat_num, on="id", how="left")
# df = df.merge(from_cat_num, on="id", how="left")

# to_cat_num = pd.read_csv("../features/symbol_graph/to_symbol_num_sum.csv")
# from_cat_num = pd.read_csv("../features/symbol_graph/from_symbol_num_sum.csv")

# df = df.merge(to_cat_num, on="id", how="left")
# df = df.merge(from_cat_num, on="id", how="left")

# to_cat_weight = pd.read_csv("../features/symbol_graph/to_symbol_weight_sum.csv")
# from_cat_weight = pd.read_csv("../features/symbol_graph/from_symbol_weight_sum.csv")

# df = df.merge(to_cat_weight, on="id", how="left")
# df = df.merge(from_cat_weight, on="id", how="left")

# to_cat_num = pd.read_csv("../features/symbol_graph/to_symbol_mean.csv")
# from_cat_num = pd.read_csv("../features/symbol_graph/from_symbol_mean.csv")

# df = df.merge(to_cat_num, on="id", how="left")
# df = df.merge(from_cat_num, on="id", how="left")

to_cat_num = pd.read_csv("../features/symbol_graph/to_symbol_num_mean.csv")
from_cat_num = pd.read_csv("../features/symbol_graph/from_symbol_num_mean.csv")

df = df.merge(to_cat_num, on="id", how="left")
df = df.merge(from_cat_num, on="id", how="left")

to_cat_weight = pd.read_csv("../features/symbol_graph/to_symbol_weight_mean.csv")
from_cat_weight = pd.read_csv("../features/symbol_graph/from_symbol_weight_mean.csv")

df = df.merge(to_cat_weight, on="id", how="left")
df = df.merge(from_cat_weight, on="id", how="left")

knn

In [81]:
# dp_knn = pd.read_csv("../features/knn/dp_knn_3.csv")
# df = df.merge(dp_knn, on="id", how="left")

# dp_knn = pd.read_csv("../features/knn/dp_knn_10.csv")
# df = df.merge(dp_knn, on="id", how="left")

# dp_knn = pd.read_csv("../features/knn/dp_knn_30.csv")
# df = df.merge(dp_knn, on="id", how="left")

# dp_knn = pd.read_csv("../features/knn/dp_knn_100.csv")
# df = df.merge(dp_knn, on="id", how="left")

# dp_knn = pd.read_csv("../features/knn/dp_knn_300.csv")
# df = df.merge(dp_knn, on="id", how="left")

中心性

In [82]:
pr = pd.read_csv("../features/graph/pagerank.csv")
df = df.merge(pr, on="id", how="left")

In [83]:
for d in ["to", "from"]:
    for w in ['', '_num', '_weight']:
        for f in ['sum', 'mean']:
            a = pd.read_csv("../features/graph/%s_pagerank%s_%s.csv" % (d, w, f))
            df = df.merge(a, on="id", how="left")

In [84]:
hits = pd.read_csv("../features/graph/hits.csv")
df = df.merge(hits, on="id", how="left")

for d in ["to", "from"]:
    for w in ['', '_num', '_weight']:
        for f in ['sum', 'mean']:
            a = pd.read_csv("../features/graph/%s_hits%s_%s.csv" % (d, w, f))
            df = df.merge(a, on="id", how="left")

二度联系人

In [85]:
# 联系人个数
for d1 in ["from", "to"]:
    for d2 in ["from", "to"]:
        for t in ["", "_num", "_weight"]:
            for f in ["sum", "mean"]:
                a = pd.read_csv("../features/graph/%s_d2_%s%s_%s.csv" % (d1, d2, t, f))
                df = df.merge(a, on="id", how="left")

In [86]:
# 二度联系人PageRank值
for d1 in ["from", "to"]:
    for d2 in ["from", "to"]:
        for t in ["num", "weight"]:
            for f in ["mean"]:
                a = pd.read_csv("../features/graph/%s_pg_%s_%s_%s_%s_%s.csv" % (d1, d2, t, f, t, f))
                df = df.merge(a, on="id", how="left")

In [87]:
# for d1 in ["from", "to"]:
#     for d2 in ["from", "to"]:
#         for t in ["num", "weight"]:
#             for f in ["sum", "mean"]:
#                 a = pd.read_csv("../features/graph/%s_hits_%s_%s_%s_%s_%s.csv" % (d1, d2, t, f, t, f))
#                 df = df.merge(a, on="id", how="left")

In [88]:
def  process_feature(train_x, valid_x, test_df):
    result = []
    drop_cols = ['id','label']
    for df in [train_x, valid_x, test_df]:
        result.append(df.drop(drop_cols, axis=1))
    return result 

In [89]:
import os
def cv(df, num_folds, param, model_dir, stratified=True, debug=False):
    if not os.path.exists(model_dir):
        os.mkdir(model_dir)
    train_df = df[df.label.notnull()]
    test_df = df[df.label.isnull()]
    seed = 178
    if "seed" in param:
        seed = param["seed"]
        del param['seed']
    
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=seed)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=seed)
        
    oof_preds = np.zeros(train_df.shape[0])
    all_test_preds = []    
    feature_importance_df = pd.DataFrame()
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df, train_df['label'])):
        train_x, train_y = train_df.iloc[train_idx], train_df['label'].iloc[train_idx]
        valid_x, valid_y = train_df.iloc[valid_idx], train_df['label'].iloc[valid_idx]
        fold_preds = test_df[["id"]]
        
        train_x, valid_x, test = process_feature(train_x, valid_x, test_df)
        if n_fold == 0:
            print(train_x.shape, valid_x.shape, test.shape)
        
        train_data = lgb.Dataset(train_x, label=train_y)
        validation_data = lgb.Dataset(valid_x, label=valid_y)

        clf=lgb.train(params,
                      train_data,
                      num_boost_round=10000,
                      valid_sets=[train_data, validation_data],
                      valid_names=["train", "valid"],
                      early_stopping_rounds=100,
                      verbose_eval=100)

        valid_preds = clf.predict(valid_x, num_iteration=clf.best_iteration)
        test_preds = clf.predict(test, num_iteration=clf.best_iteration)

        fold_preds['prob'] = test_preds
        fold_preds['fold_id'] = n_fold + 1
        all_test_preds.append(fold_preds)
        
        oof_preds[valid_idx] = valid_preds
        
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, valid_preds)))
        
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()
    full_auc = roc_auc_score(train_df['label'], oof_preds)
    print('Full AUC score %.6f' % full_auc)
    
    if not debug:
        train_df["prob"] = oof_preds
        train_df[['id', 'prob']].to_csv(model_dir + "pred_train.csv", index= False)

        all_test_preds = pd.concat(all_test_preds, axis=0)
        all_test_preds.to_csv(model_dir + "all_test_preds.csv", index=False)
        
        sub = pd.DataFrame()
        sub['id'] = all_test_preds.id.unique()
        sub.set_index("id", inplace=True)
        sub["prob"] = all_test_preds.groupby("id").prob.mean()
        sub.reset_index().to_csv(model_dir + "sub.txt", index=False)
    return full_auc

In [94]:
params_list =[
 {'boosting_type': 'goss', 'colsample_bytree': 0.8789517928114404, 'learning_rate': 0.015808125454491537, 'max_bin': 880, 'metric': 'auc', 'min_child_weight': 9.15663348426493, 'num_leaves': 129, 'reg_alpha': 46.1851872725305, 'reg_lambda': 694.9478428106884, 'subsample': 1.0, 'verbose': 1, 'seed': 6580},
 {'boosting_type': 'goss', 'colsample_bytree': 0.8557602070454506, 'learning_rate': 0.015982317941686752, 'max_bin': 800, 'metric': 'auc', 'min_child_weight': 5.564230550095491, 'num_leaves': 118, 'reg_alpha': 24.840163468707626, 'reg_lambda': 633.4853189068382, 'subsample': 1.0, 'verbose': 1, 'seed': 4997},
 {'boosting_type': 'goss', 'colsample_bytree': 0.6298338994793943, 'learning_rate': 0.01859568652777303, 'max_bin': 840, 'metric': 'auc', 'min_child_weight': 0.333952927570472, 'num_leaves': 60, 'reg_alpha': 32.59439665622528, 'reg_lambda': 814.9101871401995, 'subsample': 1.0, 'verbose': 1, 'seed': 4069},
 {'boosting_type': 'goss', 'colsample_bytree': 0.8687058739655211, 'learning_rate': 0.01847363528403374, 'max_bin': 780, 'metric': 'auc', 'min_child_weight': 1.9753708845666633, 'num_leaves': 132, 'reg_alpha': 42.60449928039026, 'reg_lambda': 697.9037863096568, 'subsample': 1.0, 'verbose': 1, 'seed': 9779},
 {'boosting_type': 'goss', 'colsample_bytree': 0.7826797501564828, 'learning_rate': 0.017533025541933332, 'max_bin': 760, 'metric': 'auc', 'min_child_weight': 14.13195762616, 'num_leaves': 91, 'reg_alpha': 26.59764529883372, 'reg_lambda': 783.0357736651754, 'subsample': 1.0, 'verbose': 1, 'seed': 6379}
]
result = []
for i, params in enumerate(params_list):
    model_dir = "../model_output/random/lgb_628/%d/" % i
    result.append(cv(df, 5, params, model_dir))

(15166, 628) (3793, 628) (6000, 628)
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.835142	valid's auc: 0.653671
[200]	train's auc: 0.892334	valid's auc: 0.672952
[300]	train's auc: 0.924587	valid's auc: 0.681574
[400]	train's auc: 0.943635	valid's auc: 0.68718
[500]	train's auc: 0.957435	valid's auc: 0.692288
[600]	train's auc: 0.967129	valid's auc: 0.695602
[700]	train's auc: 0.973927	valid's auc: 0.695587
[800]	train's auc: 0.980036	valid's auc: 0.69757
[900]	train's auc: 0.983828	valid's auc: 0.696633
Early stopping, best iteration is:
[807]	train's auc: 0.980288	valid's auc: 0.69786


/home/LAB/yanhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/LAB/yanhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Fold  1 AUC : 0.697860
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.837421	valid's auc: 0.654547
[200]	train's auc: 0.897378	valid's auc: 0.680421
[300]	train's auc: 0.927408	valid's auc: 0.689837
[400]	train's auc: 0.946239	valid's auc: 0.692704
Early stopping, best iteration is:
[386]	train's auc: 0.944123	valid's auc: 0.693418
Fold  2 AUC : 0.693418
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.777418	valid's auc: 0.667252
[200]	train's auc: 0.830663	valid's auc: 0.692512
[300]	train's auc: 0.864063	valid's auc: 0.701396
[400]	train's auc: 0.877781	valid's auc: 0.707042
[500]	train's auc: 0.900603	valid's auc: 0.710353
[600]	train's auc: 0.921972	valid's auc: 0.714009
[700]	train's auc: 0.929762	valid's auc: 0.71443
[800]	train's auc: 0.941934	valid's auc: 0.714323
Early stopping, best iteration is:
[769]	train's auc: 0.93842	valid's auc: 0.715103
Fold  3 AUC : 0.715103
Training until validation scores don'

/home/LAB/yanhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(15166, 628) (3793, 628) (6000, 628)
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.876863	valid's auc: 0.647126
[200]	train's auc: 0.931048	valid's auc: 0.671674
[300]	train's auc: 0.957684	valid's auc: 0.683798
[400]	train's auc: 0.974072	valid's auc: 0.684903
[500]	train's auc: 0.984298	valid's auc: 0.688781
[600]	train's auc: 0.990408	valid's auc: 0.691967
[700]	train's auc: 0.99401	valid's auc: 0.69279
[800]	train's auc: 0.995838	valid's auc: 0.695127
Early stopping, best iteration is:
[756]	train's auc: 0.994971	valid's auc: 0.695484
Fold  1 AUC : 0.695484
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.882734	valid's auc: 0.651927
[200]	train's auc: 0.932195	valid's auc: 0.667218
[300]	train's auc: 0.958085	valid's auc: 0.673299
[400]	train's auc: 0.973305	valid's auc: 0.677441
[500]	train's auc: 0.98321	valid's auc: 0.679798
[600]	train's auc: 0.988725	valid's auc: 0.68085
Early stopping, best iteration is

[900]	train's auc: 0.990619	valid's auc: 0.711541
Early stopping, best iteration is:
[821]	train's auc: 0.988375	valid's auc: 0.712347
Fold  5 AUC : 0.712347
Full AUC score 0.701629
(15166, 628) (3793, 628) (6000, 628)
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.867088	valid's auc: 0.669407
[200]	train's auc: 0.922122	valid's auc: 0.692182
[300]	train's auc: 0.950453	valid's auc: 0.698877
[400]	train's auc: 0.968167	valid's auc: 0.702705
[500]	train's auc: 0.979253	valid's auc: 0.704154
Early stopping, best iteration is:
[493]	train's auc: 0.978942	valid's auc: 0.704366
Fold  1 AUC : 0.704366
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.865988	valid's auc: 0.683899
[200]	train's auc: 0.915184	valid's auc: 0.701435
[300]	train's auc: 0.948974	valid's auc: 0.708484
[400]	train's auc: 0.961244	valid's auc: 0.713277
[500]	train's auc: 0.974227	valid's auc: 0.71491
Early stopping, best iteration is:
[456]	train's

In [95]:
result

[0.70263969065941634,
 0.70222563110036418,
 0.70165881086583792,
 0.70162871412071937,
 0.70140848666486488]

In [91]:
# params_list =[
#  {'boosting_type': 'goss', 'colsample_bytree': 0.6763559283988655, 'learning_rate': 0.01825380539267228, 'max_bin': 660, 'metric': 'auc', 'min_child_weight': 10.799395853190386, 'num_leaves': 168, 'reg_alpha': 33.17114637032081, 'reg_lambda': 790.9627167813759, 'subsample': 1.0, 'verbose': 1},
#  {'boosting_type': 'goss', 'colsample_bytree': 0.7685951652752658, 'learning_rate': 0.021662727403280953, 'max_bin': 510, 'metric': 'auc', 'min_child_weight': 9.601137206110081, 'num_leaves': 189, 'reg_alpha': 42.34282656577136, 'reg_lambda': 632.1634663301063, 'subsample': 1.0, 'verbose': 1},
#  {'boosting_type': 'goss', 'colsample_bytree': 0.7621940775240458, 'learning_rate': 0.02230476487346519, 'max_bin': 590, 'metric': 'auc', 'min_child_weight': 18.638341031245165, 'num_leaves': 156, 'reg_alpha': 40.67124722501198, 'reg_lambda': 827.062292850617, 'subsample': 1.0, 'verbose': 1},
#  {'boosting_type': 'goss', 'colsample_bytree': 0.7848040606199326, 'learning_rate': 0.019545851131539383, 'max_bin': 610, 'metric': 'auc', 'min_child_weight': 15.751419432526315, 'num_leaves': 154, 'reg_alpha': 37.71019047359832, 'reg_lambda': 771.3021799497349, 'subsample': 1.0, 'verbose': 1},
#  {'boosting_type': 'goss', 'colsample_bytree': 0.7769871598585915, 'learning_rate': 0.0225987638086461, 'max_bin': 680, 'metric': 'auc', 'min_child_weight': 16.799690425689057, 'num_leaves': 200, 'reg_alpha': 46.95522776917207, 'reg_lambda': 858.5739765649753, 'subsample': 1.0, 'verbose': 1}
# ]
# result = []
# for i, params in enumerate(params_list):
#     model_dir = "../model_output/lgb_628/%d/" % i
#     result.append(cv(df, 5, params, model_dir))

In [92]:
# result

[0.70113447571038345,
 0.69992062401635224,
 0.69906313116565111,
 0.69829710479455709,
 0.69723567924781837]